# Chapter 2: Networking & System Fundamentals for Security

In Chapter 1, we established the strategic foundations of cybersecurity—the *why* and the *what*. Now, we must master the *where*. Before you can secure an application, you must understand the terrain it operates within: the networks that carry its data, the operating systems that host its processes, and the virtualized environments that define modern infrastructure.

This chapter bridges the gap between abstract security principles and tangible technical implementation. We will traverse the OSI model, dissect the protocols that power the internet, harden operating systems against compromise, and explore the security implications of containerization. By the end, you will possess the architectural literacy required to design systems that are secure by design, not by accident.

---

## 2.1 Networking Refresher: OSI/TCP-IP Models, Protocols, and Ports

Networks are the nervous system of digital infrastructure. To secure them, we must understand how data moves, where it can be intercepted, and how protocols can be exploited.

### 2.1.1 The OSI and TCP/IP Models

The **Open Systems Interconnection (OSI)** model conceptualizes network communication in seven layers. While the modern internet runs on the simplified **TCP/IP model** (4 layers), security professionals use the OSI model to pinpoint where attacks occur.

| OSI Layer | TCP/IP Layer | Function | Security Relevance |
|-----------|--------------|----------|-------------------|
| 7. Application | Application | HTTP, FTP, DNS, SMTP | SQL Injection, XSS, Session Hijacking |
| 6. Presentation | Application | TLS/SSL, Encryption | Protocol downgrade attacks |
| 5. Session | Application | NetBIOS, RPC | Session fixation, hijacking |
| 4. Transport | Transport | TCP, UDP | SYN floods, Port scanning |
| 3. Network | Internet | IP, ICMP, Routing | IP spoofing, DDoS, MITM |
| 2. Data Link | Network Interface | Ethernet, MAC addresses | ARP poisoning, MAC flooding |
| 1. Physical | Network Interface | Cables, Radio signals | Wiretapping, Faraday cages |

**Key Insight for Developers:** Most application-layer vulnerabilities (your code) interact with Layer 7, but attacks often originate at lower layers. A **Man-in-the-Middle (MITM)** attack at Layer 2 (ARP poisoning) can compromise your Layer 7 HTTPS traffic if certificate validation is improperly implemented.

### 2.1.2 Critical Protocols and Their Security Profiles

#### **HTTP/HTTPS (Ports 80/443)**
*   **HTTP (80):** Unencrypted. Never transmit sensitive data over HTTP in 2026.
*   **HTTPS (443):** HTTP over TLS 1.3 (the current standard). TLS 1.3 removes legacy algorithms vulnerable to downgrade attacks.

**Configuration Snippet: Enforcing TLS 1.3 (Nginx)**
```nginx
server {
    listen 443 ssl http2;
    server_name secureapp.example.com;
    
    # Only allow strong protocols and ciphers
    ssl_protocols TLSv1.3;
    ssl_ciphers 'TLS_AES_256_GCM_SHA384:TLS_CHACHA20_POLY1305_SHA256';
    ssl_prefer_server_ciphers off;
    
    # HTTP Strict Transport Security (HSTS)
    add_header Strict-Transport-Security "max-age=31536000; includeSubDomains" always;
}
```

#### **DNS (Port 53)**
The Domain Name System translates human-readable names to IP addresses. It is inherently insecure.
*   **DNS Spoofing/Poisoning:** Attackers inject false DNS records to redirect traffic.
*   **DNSSEC (DNS Security Extensions):** Cryptographically verifies DNS responses.
*   **DoH/DoT (DNS over HTTPS/TLS):** Encrypts DNS queries to prevent eavesdropping.

**Diagnostic Command: Checking DNS Configuration**
```bash
# Query DNS records with security details
dig +dnssec example.com

# Check for DNSSEC validation
delv example.com
```

#### **SMTP (Ports 25, 587, 465)**
Email remains a primary attack vector for phishing and malware.
*   **Port 25:** Legacy SMTP (often blocked by ISPs to prevent spam).
*   **Port 587:** Modern submission with STARTTLS (upgrades plain to encrypted).
*   **Port 465:** SMTPS (implicit TLS).
*   **SPF, DKIM, DMARC:** Email authentication protocols to prevent spoofing.

#### **SSH (Port 22)**
Secure Shell is the standard for remote administration but is constantly brute-forced.
*   **Best Practice:** Disable password authentication; use key-based authentication exclusively.

**Configuration Snippet: Hardening SSH (`/etc/ssh/sshd_config`)**
```bash
# Disable root login
PermitRootLogin no

# Use key-based authentication only
PasswordAuthentication no
PubkeyAuthentication yes

# Only allow specific users
AllowUsers deploy_admin backup_service

# Change default port (security through obscurity - minor deterrent)
Port 2222

# Use only strong algorithms
Ciphers chacha20-poly1305@openssh.com,aes256-gcm@openssh.com
MACs hmac-sha2-512-etm@openssh.com,hmac-sha2-256-etm@openssh.com
```

### 2.1.3 Ports and Scanning
Ports are logical endpoints for communication.
*   **Well-known ports:** 0-1023 (require root/admin to bind).
*   **Ephemeral ports:** 1024-65535 (temporary client-side ports).

**Educational Snippet: Port Scanning with Nmap**
Understanding how attackers scan helps you defend. This command identifies open services and their versions (banner grabbing):

```bash
# SYN stealth scan (requires root)
sudo nmap -sS -sV -O --script vuln target.example.com

# Explanation:
# -sS: SYN scan (half-open, less logging)
# -sV: Version detection
# -O: OS fingerprinting
# --script vuln: Run vulnerability detection scripts
```

**Defense:** Run `netstat` or `ss` regularly on your servers to audit listening ports:
```bash
# Show all listening TCP sockets with process info
ss -tulnp | grep LISTEN
```

---

## 2.2 Network Security Basics: Firewalls, IDS/IPS, VPNs, and Segmentation

Once you understand traffic flow, you must implement controls to filter it.

### 2.2.1 Firewalls: The First Line of Defense
Firewalls enforce rules based on IP addresses, ports, and protocols.

**Types:**
1.  **Stateless:** Examines individual packets against ACLs (Access Control Lists). Fast but dumb.
2.  **Stateful:** Tracks the state of active connections. Understands that an outgoing HTTP request (port 443) should allow the return traffic.
3.  **Application Layer (WAF):** Inspects HTTP/HTTPS traffic for SQLi, XSS (Chapter 10).

**Implementation: iptables (Linux)**
`iptables` is the traditional Linux firewall. Modern systems use `nftables`, but `iptables` syntax remains widespread.

```bash
# Flush existing rules (careful in production!)
iptables -F

# Default policy: Drop everything
iptables -P INPUT DROP
iptables -P FORWARD DROP
iptables -P OUTPUT ACCEPT

# Allow established connections
iptables -A INPUT -m conntrack --ctstate ESTABLISHED,RELATED -j ACCEPT

# Allow loopback (essential for local services)
iptables -A INPUT -i lo -j ACCEPT

# Allow SSH (port 2222 if you changed it)
iptables -A INPUT -p tcp --dport 2222 -j ACCEPT

# Allow HTTP/HTTPS
iptables -A INPUT -p tcp -m multiport --dports 80,443 -j ACCEPT

# Drop invalid packets
iptables -A INPUT -m conntrack --ctstate INVALID -j DROP

# Save rules (Debian/Ubuntu)
iptables-save > /etc/iptables/rules.v4
```

### 2.2.2 IDS/IPS: Detection vs. Prevention
*   **IDS (Intrusion Detection System):** Monitors and alerts (passive). Think burglar alarm.
*   **IPS (Intrusion Prevention System):** Monitors and blocks (active). Think automated turret.
*   **NIDS vs HIDS:** Network-based (monitors traffic) vs Host-based (monitors logs/files).

**Open Source Example: Snort/Suricata**
These tools use **signatures** (known attack patterns) and **anomaly detection** (statistical deviations).

### 2.2.3 VPNs: Encrypted Tunnels
Virtual Private Networks create encrypted tunnels over public networks.
*   **Site-to-Site:** Connects two networks (e.g., HQ to Data Center) using IPsec.
*   **Remote Access:** Individual users connecting (OpenVPN, WireGuard, IPSec).
*   **Zero Trust:** Modern replacement for VPNs. Assumes breach; verifies every request regardless of network location (covered in Chapter 18).

### 2.2.4 Network Segmentation
**Flat networks** (everything on one subnet) are dangerous. If an attacker breaches the web server, they can reach the database.

**Segmentation Strategies:**
1.  **VLANs (Virtual LANs):** Logically separate networks at Layer 2.
    *   VLAN 10: Web Servers (DMZ)
    *   VLAN 20: Application Servers
    *   VLAN 30: Database Servers
    *   VLAN 99: Management/Admin
2.  **DMZ (Demilitarized Zone):** A buffer zone between the internet and internal network. Public-facing servers live here.
3.  **Micro-segmentation:** Cloud-native approach using software-defined perimeters (e.g., AWS Security Groups, Kubernetes Network Policies).

**Subnetting Basics:**
Understanding CIDR notation (e.g., `192.168.1.0/24`) is crucial for firewall rules. The `/24` indicates the subnet mask, defining the network size.

---

## 2.3 Operating System Security: Windows & Linux Hardening

Your application runs on an OS. If the OS is compromised, your application is compromised, regardless of how secure your code is.

### 2.3.1 Linux Security Hardening
Linux powers ~90% of public cloud servers. Securing it follows the **CIS (Center for Internet Security) Benchmarks**, the industry gold standard.

**1. File Permissions and Ownership**
Linux uses Discretionary Access Control (DAC) via `rwx` (read/write/execute) bits.

```bash
# View permissions
ls -la /var/www/html
# Output: -rw-r--r-- 1 www-data www-data index.html

# Meaning:
# - : File type (- file, d directory, l link)
# rw- : Owner (www-data) can read/write
# r-- : Group can only read
# r-- : Others can only read

# Secure a file (owner rw, group r, others nothing)
chmod 640 sensitive_config.yml

# Change ownership
sudo chown root:admin backup_script.sh
```

**2. Mandatory Access Control (MAC): SELinux/AppArmor**
DAC is often insufficient. MAC systems enforce policies that even root cannot override.
*   **SELinux (Red Hat/CentOS):** Enforces strict policies; can be complex.
*   **AppArmor (Debian/Ubuntu):** Profile-based, easier to configure.

```bash
# Check SELinux status
getenforce
# Enforcing = Active, Permissive = Logging only, Disabled = Off

# View AppArmor status
aa-status
```

**3. Service Management (systemd)**
Disable unnecessary services to reduce attack surface.

```bash
# List active services
systemctl list-units --type=service --state=running

# Disable a service (e.g., CUPS printing service on a server)
sudo systemctl stop cups
sudo systemctl disable cups

# Check service for security settings
systemctl show sshd | grep -i security
```

**4. Log Monitoring**
Linux uses `syslog` or `journald`. Critical logs include:
*   `/var/log/auth.log` (authentication attempts)
*   `/var/log/syslog` (general system)
*   `/var/log/audit/audit.log` (SELinux/AppArmor)

**Tool: Fail2Ban**
Bans IPs after multiple failed login attempts:
```bash
# Install
sudo apt install fail2ban

# Configure jail for SSH (/etc/fail2ban/jail.local)
[sshd]
enabled = true
port = 2222
filter = sshd
logpath = /var/log/auth.log
maxretry = 3
bantime = 3600
```

### 2.3.2 Windows Security Hardening
Windows Server and client OSes dominate enterprise environments.

**1. User Account Control (UAC) and Privileges**
*   **Principle of Least Privilege:** Users should run as Standard Users, not Administrators.
*   **Local Administrator Password Solution (LAPS):** Randomizes local admin passwords to prevent lateral movement.

**2. Windows Defender & AMSI**
*   **AMSI (Antimalware Scan Interface):** Allows Windows Defender to scan scripts (PowerShell, VBScript, JScript) in memory, catching fileless malware.

**3. PowerShell Security**
PowerShell is a double-edged sword: powerful for admins, dangerous for attackers.

```powershell
# Check execution policy (should be RemoteSigned or AllSigned, not Unrestricted)
Get-ExecutionPolicy

# Enable script block logging (captures full script content)
Set-ItemProperty -Path "HKLM:\Software\Policies\Microsoft\Windows\PowerShell\ScriptBlockLogging" -Name "EnableScriptBlockLogging" -Value 1

# Enable transcription (logs all input/output)
Set-ItemProperty -Path "HKLM:\Software\Policies\Microsoft\Windows\PowerShell\Transcription" -Name "EnableTranscripting" -Value 1
```

**4. Windows Firewall (Host-based)**
```powershell
# Enable firewall on all profiles
Set-NetFirewallProfile -Profile Domain,Public,Private -Enabled True

# Block inbound by default, allow outbound
Set-NetFirewallProfile -DefaultInboundAction Block -DefaultOutboundAction Allow

# Create a rule to allow RDP only from specific subnet
New-NetFirewallRule -DisplayName "Restricted RDP" -Direction Inbound -Protocol TCP -LocalPort 3389 -RemoteAddress 192.168.1.0/24 -Action Allow
```

### 2.3.3 Patch Management
Unpatched systems are the #1 initial access vector.
*   **Linux:** `unattended-upgrades` (Debian), `dnf-automatic` (RHEL).
*   **Windows:** WSUS (Windows Server Update Services) or Intune/Endpoint Manager.
*   **Emergency Patching:** When a **0-day** (unpatched vulnerability) drops, speed is critical.

---

## 2.4 Virtualization & Containers: Security Implications

Modern infrastructure is rarely bare-metal. Understanding the security boundaries (and escapes) of virtualized environments is essential.

### 2.4.1 Virtual Machines (VMs) and Hypervisors
*   **Type 1 (Bare-metal):** VMware ESXi, Hyper-V, Xen. More secure; OS is minimal.
*   **Type 2 (Hosted):** VMware Workstation, VirtualBox. Less secure; runs on top of host OS.

**Security Concerns:**
1.  **VM Escape:** An attacker breaks out of the VM to access the host or other VMs. Rare but catastrophic (e.g., VMware CVEs).
2.  **VM Sprawl:** Unused VMs left running become unmanaged vulnerabilities.
3.  **Snapshot Security:** Snapshots contain memory dumps (potential credential exposure) and should be encrypted.

### 2.4.2 Docker Security
Containers share the host kernel but are isolated via **namespaces** (isolation) and **cgroups** (resource limits). However, "isolated" ≠ "secure."

**1. Image Security**
*   **Minimal Base Images:** Use `distroless` or `alpine` images to reduce attack surface.
*   **Scanning:** Scan images for CVEs before deployment.

**Secure Dockerfile Example:**
```dockerfile
# Multi-stage build: Compile in full environment, run in minimal
FROM golang:1.21 AS builder
WORKDIR /app
COPY . .
RUN CGO_ENABLED=0 GOOS=linux go build -o main

# Use distroless for final image (no shell, no package manager)
FROM gcr.io/distroless/static-debian12
WORKDIR /app
COPY --from=builder /app/main .

# Run as non-root user ( distroless uses nonroot by default, uid 65532)
USER nonroot:nonroot

EXPOSE 8080
ENTRYPOINT ["/app/main"]
```

**2. Container Runtime Security**
*   **Privileged Mode:** Never run containers with `--privileged`. It grants full host access.
*   **Read-Only Filesystems:** Prevent attackers from writing malware to the container.

```bash
# Run with security options
docker run -d \
  --read-only \
  --security-opt=no-new-privileges:true \
  --cap-drop=ALL \
  --cap-add=NET_BIND_SERVICE \
  -p 8080:8080 \
  myapp:latest
```

**3. The Docker Socket Risk**
Mounting `/var/run/docker.sock` into a container is dangerous; it grants container management access to the host.

### 2.4.3 Kubernetes Basics (K8s)
Kubernetes orchestrates containers at scale. Security in K8s is a vast topic (covered in Chapter 13), but fundamentals include:

**1. Pod Security Standards (PSS)**
Replaced the deprecated Pod Security Policies. Enforces three levels:
*   **Privileged:** Unrestricted (dangerous).
*   **Baseline:** Prevents known privilege escalations.
*   **Restricted:** Heavily locked down (production standard).

**2. RBAC (Role-Based Access Control)**
Restricts what Kubernetes API resources a user or service account can access.

```yaml
# Example: Role allowing only read access to pods in a specific namespace
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  namespace: production
  name: pod-reader
rules:
- apiGroups: [""]
  resources: ["pods"]
  verbs: ["get", "watch", "list"]
```

**3. Network Policies**
By default, all pods in a cluster can communicate. Network Policies act as firewalls between pods.

```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: db-isolation
spec:
  podSelector:
    matchLabels:
      app: database
  policyTypes:
  - Ingress
  ingress:
  - from:
    - podSelector:
        matchLabels:
          app: frontend
    ports:
    - protocol: TCP
      port: 5432
```

---

### Chapter Summary

In this chapter, we mapped the **OSI model** to understand where attacks occur, configured **firewalls** to filter traffic, and implemented **OS hardening** on both Linux and Windows systems. We explored the critical differences between **VMs** and **containers**, learning that while Docker provides process isolation, it requires specific configurations (non-root users, read-only filesystems, capability dropping) to be truly secure. Finally, we touched upon **Kubernetes**, establishing that orchestration security relies on RBAC and Network Policies to enforce Zero Trust within the cluster.

These networking and system fundamentals provide the infrastructure layer upon which all secure applications are built. However, infrastructure alone cannot protect data in transit or at rest. To ensure **Confidentiality** and **Integrity** across these networks, we must master the mathematical and practical aspects of cryptography—the science of secret writing.

**Next Up: Chapter 3: Cryptography Essentials**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='1. the_cybersecurity_landscape.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='3. cryptography_essentials.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
